In [34]:
%matplotlib widget
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt



#数据读取与质量预处理
data = io.loadmat('../../Matlab_files/top3d_Model_9/topModel_9.mat')
data = data['data']
data = np.array(data)
print('数据初始维度：',data.shape)
data = data.transpose(1, 2, 0)
print('数据调整后维度：',data.shape)#(x,z,y)
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] > 0.5:
                data[i][j][k] = 1
            else:
                data[i][j][k] = 0



#数据尺寸扩展为标准尺寸
nelx, nely, nelz = 120, 120, 120
if (nelx - data.shape[0])%2 !=0:
    data = np.pad(data, ((1, 0), (0, 0), (0 ,0)))
if (nelz - data.shape[1])%2 !=0:
    data = np.pad(data, ((0, 0), (1, 0), (0 ,0)))
if (nely - data.shape[2])%2 !=0:
    data = np.pad(data, ((0, 0), (0, 0), (1 ,0)))
data_new = np.pad(data, (((nelx - data.shape[0])//2, (nelx - data.shape[0])//2),
                    ((nely - data.shape[1])//2, (nely - data.shape[1])//2),
                    ((nelz - data.shape[2])//2, (nelx - data.shape[2])//2)))
print('数据标准化后维度：',data_new.shape)



#尺寸标准化后模型可视化
#测试保存
# io.savemat('C:/Users/11573/Desktop/data/standardinizetest.mat',{'standarddata':data_new})
#正式保存
io.savemat('../../Matlab_files/top3d_Model_9/standard_topModel_9.mat',{'standarddata':data_new})
# mycolormap = plt.get_cmap('winter')
# data_new_max = data_new.max()
# relativevalue = np.round(data_new / data_new_max, 1)
# colorsvalues = mycolormap(relativevalue)
# fig = plt.figure(figsize=(15, 15))
# ax = fig.add_subplot(projection='3d')
# ax.set_xlabel('X Axis')
# ax.set_ylabel('Y Axis')
# ax.set_zlabel('Z Axis')
# ax.set_xlim(0, 150)
# ax.set_ylim(0, 150)
# ax.set_zlim(75, -75)
# ax.voxels(data_new, facecolors=colorsvalues,edgecolors = 'k', shade = False)


#查找模型存在的单共享边等断裂结构
defect_data = np.zeros((120, 120, 120))
face_voxel = []
search_list = np.zeros((1, 6, 3))
n = 0
t = 0
l = 0
defect_model = np.zeros((nely, nelx, nelz))
defect_index = np.array([[0, 0, 0]])
for i in range(1,(data_new.shape[0]-1)):
    for j in range(1,(data_new.shape[1]-1)):
        for k in range(1,(data_new.shape[2]-1)):
            if data_new[i][j][k] !=0:
                face_voxel.append(data_new[i-1][j][k])
                # print('1:',face_voxel)
                face_voxel.append(data_new[i+1][j][k])
                # print('2:',face_voxel)
                face_voxel.append(data_new[i][j-1][k])
                # print('3:',face_voxel)
                face_voxel.append(data_new[i][j+1][k])
                # print('4:',face_voxel)
                face_voxel.append(data_new[i][j][k-1])
                # print('5:',face_voxel)
                face_voxel.append(data_new[i][j][k+1])
                # print('6:',face_voxel)
                sum_value = sum(face_voxel)
                # print('face_voxel:',face_voxel)
                
                if sum_value == 0 or sum_value == 1:
                    x, y, z = i, j, k
                    defect_array = np.array([[x, y ,z]])
                    # print('i,j,k:',i, j, k)
                    defect_index = np.append(defect_index, defect_array, axis = 0)
                    
                    # defect_index = np.array(defect_index)
                    # print(defect_index.shape)
                    defect_data[i-2:i+2,j-2:j+2,k-2:k+2] = data_new[i-2:i+2,j-2:j+2,k-2:k+2]
                    face_voxel_index = np.array([[[x-1, z, y],
                                                 [x+1, z, y],
                                                 [x, z-1, y],
                                                 [x, z+1, y],
                                                 [x, z, y-1],
                                                 [x, z, y+1]]])
                    search_list = np.append(search_list, face_voxel_index, axis=0)
                    l += 1
                   
                face_voxel = []
                t += 1    
            n += 1

#制作在matlab中显示的缺陷中心点
for t1 in range(1, len(defect_index)):
    exam_index = defect_index[t1]
    x1, y1, z1 = exam_index[0], exam_index[1], exam_index[2]
    defect_model[x1][y1][z1] = 1
defect_data = defect_data + defect_model
# Top3dSTL_v3('C:\Users\11573\Desktop\VSCode_repos\Generate_dataset\Matlab_files\top3d_Model_1\repairtopModel_1.STL',repairdata,'Format','binary','Mode','iso')
#修复模型

data_new[44][59][67] = 1
data_new[44][60][67] = 1
data_new[47][59][68] = 1
data_new[47][60][68] = 1
data_new[49][59][69] = 1
data_new[49][60][69] = 1
data_new[41][60][63] = 1
data_new[41][59][63] = 1
data_new[65][52][59] = 1
data_new[65][67][59] = 1
data_new[67][53][57] = 1
data_new[67][66][57] = 1
data_new[71][57][55] = 1
data_new[71][62][55] = 1
data_new[70][57][56] = 1
data_new[70][62][56] = 1
data_new[73][58][54] = 1
data_new[73][61][54] = 1
data_new[72][58][55] = 1
data_new[72][61][55] = 1
data_new[73][56][65] = 1
data_new[73][63][65] = 1
data_new[75][56][65] = 1
data_new[75][63][65] = 1
data_new[75][58][66] = 1
data_new[75][61][66] = 1
data_new[76][57][65] = 1
data_new[76][62][65] = 1
data_new[77][57][66] = 1
data_new[77][62][66] = 1
data_new[77][57][52] = 1
data_new[77][62][52] = 1
data_new[79][57][52] = 1
data_new[79][62][52] = 1
data_new[76][59][68] = 1
data_new[76][60][68] = 1
data_new[77][59][51] = 1
data_new[77][60][51] = 1
# data_new[68][61][59] = 1
# data_new[68][61][62] = 1
# data_new[69][61][61] = 1
# data_new[69][61][64] = 1
# data_new[70][61][63] = 1
# data_new[70][61][66] = 1
# data_new[71][61][65] = 1
# data_new[71][61][68] = 1
# data_new[71][61][69] = 1
# data_new[72][61][67] = 1
# data_new[72][61][69] = 1
# data_new[73][61][67] = 1
# data_new[73][61][69] = 1
# data_new[74][61][68] = 1
# data_new[74][61][69] = 1
# data_new[75][61][66] = 1
# data_new[77][61][65] = 1
# data_new[79][61][64] = 1
# data_new[81][61][63] = 1
# data_new[83][61][63] = 1
print(defect_index)
# print('遍历了：',n,'次')
# print('共有',t,'个非零元素')
# print('共有',l,'个缺陷点')
# print('search_list:',search_list)
# print('search_list.shape:',search_list.shape)


# # #针对模型缺陷部分进行插补
# for o in range(1,len(search_list)):
#     repair_index = search_list[o]
#     for w in range(6):
#         point_index = repair_index[w]
#         index_x = int(point_index[0])
#         index_z = int(point_index[1])
#         index_y = int(point_index[2])
#         data_new[index_x, index_z, index_y] = 1

#测试保存
# io.savemat('C:/Users/11573/Desktop/data/defecttopModel.mat',{'defectdata':defect_data})

# #正式保存
io.savemat('../../Matlab_files/top3d_Model_9/defecttopModel_9.mat',{'defectdata':defect_data})
io.savemat('../../Matlab_files/top3d_Model_9/repairtopModel_9.mat',{'repairdata':data_new})

# # #抓取的缺陷结构三维可视化
# mycolormap = plt.get_cmap('winter')
# defect_data_max = defect_data.max()
# relativevalue = np.round(defect_data / defect_data_max, 1)
# colorsvalues = mycolormap(relativevalue)
# fig = plt.figure(figsize=(15, 15))
# ax = fig.add_subplot(projection='3d')
# ax.set_xlabel('X Axis')
# ax.set_ylabel('Y Axis')
# ax.set_zlabel('Z Axis')
# ax.set_xlim(0, 150)
# ax.set_ylim(0, 150)
# ax.set_zlim(75, -75)
# ax.voxels(defect_data, facecolors=colorsvalues,edgecolors = 'k', shade = False)

# # #修补后整体结构三维可视化
# mycolormap = plt.get_cmap('winter')
# data_new_max = data_new.max()
# relativevalue = np.round(data_new / data_new_max, 1)
# colorsvalues = mycolormap(relativevalue)
# fig = plt.figure(figsize=(15, 15))
# ax = fig.add_subplot(projection='3d')
# ax.set_xlabel('X Axis')
# ax.set_ylabel('Y Axis')
# ax.set_zlabel('Z Axis')
# ax.set_xlim(0, 150)
# ax.set_ylim(0, 150)
# ax.set_zlim(75, -75)
# ax.voxels(data_new, facecolors=colorsvalues,edgecolors = 'k', shade = False)


数据初始维度： (20, 50, 16)
数据调整后维度： (50, 16, 20)
数据标准化后维度： (120, 120, 120)
[[ 0  0  0]
 [42 59 63]
 [42 60 63]
 [65 52 58]
 [65 67 58]
 [68 53 57]
 [68 66 57]
 [71 57 56]
 [71 62 56]
 [73 58 55]
 [73 61 55]
 [74 56 65]
 [74 63 65]
 [76 57 66]
 [76 62 66]
 [78 57 52]
 [78 62 52]]
